# Import library

In [ ]:
!pip install PySastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

2.5.0


# Function to stem and stopword removal

In [ ]:
stop = []
with  open('/content/drive/MyDrive/Bangkit/stopword.txt') as f:
    csv_file = csv.reader(f)
    for word in csv_file:
        stop.append(word[0])

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
print(stop)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [ ]:
dictionary = ArrayDictionary(stop)
stopword = StopWordRemover(dictionary)

# Preprocess Dataset

## Stem and Stopword removal

In [ ]:
dataset = pd.read_csv(r'/content/drive/MyDrive/Bangkit/new_news.csv')
dataset['berita_preprocessed'] = dataset.apply(lambda x:  stopword.remove(stemmer.stem(x['berita'])), axis=1)
# dataset['class'] = dataset['tagging'].replace({'Valid':1, 'Hoax':0})
dataset.drop_duplicates(subset='berita_preprocessed', inplace=True)
dataset.drop(columns=['berita', 'tagging'], inplace=True)
dataset
# dataset.to_csv(path_or_buf='/content/preprocessed.csv',index=False)

TypeError: ignored

## Split dataset to train, val, and test set (80, 10, 10)

In [ ]:
trainset = dataset.sample(frac=0.8).reset_index(drop=True)
print(trainset.head(1))

                                 berita_preprocessed  class
0  menteri komunikasi informatika rudiantara kont...      0


In [ ]:
trainset.to_csv(path_or_buf='/content/trainset.csv',index=False)

In [ ]:
dataset = dataset.merge(trainset, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].drop(columns=['_merge'])
valset = dataset.sample(frac=0.5).reset_index(drop=True)
print(valset)

                                  berita_preprocessed  class
0   badan narkotika nasional bnn rampung proses pe...      1
1   masyakarat surabaya jawa timur heboh cuat isu ...      1
2   stroke ganggu buluh darah otak ganggu upa sumb...      1
3   lele rupa ikan habitat air tawar ikan ternak l...      0
4   masyarakat percaya tusuk jarum daun telinga to...      1
5   hidup aneka kuas rias wajah luk bersih debu ca...      0
6   sikat gigi rupa alat mandi hilang kotor gigi s...      0
7   aparat polisi polda lampung laku lidi antisipa...      1
8   gaji presiden ri drastis capai rp 500 juta rp ...      0
9   facebook tutup 24 april 2018 kemkominfo netize...      1
10  kabar permen dot indikasi kandung narkoba sura...      0
11  foto aksi 212 jilid 2 masuk masjid istiqlal pa...      1
12  tusuk jarum telinga pasien stroke cegah darah ...      1
13  aple buka suara kena iphone 6 lengkung ramai b...      1
14  sebar kabar media sosial tahan teroris rumah t...      0
15  massa daerah kamis 1

In [ ]:
valset.to_csv(path_or_buf='/content/valset.csv',index=False)

In [ ]:
testset = dataset.merge(valset, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].drop(columns=['_merge']).reset_index(drop=True)
print(testset)

                                  berita_preprocessed  class
0   isu ikan lele kandung sel kanker jejaring soci...      1
1   clarias ikan lele topik bincang hangat putar d...      1
2   lele rupa salah jenis ikan favorit saji menu m...      1
3   lele ikan rupa favorit orang milik enak milik ...      0
4   lele rupa ikan habitat air tawar ikan ternak l...      0
5   lele salah bahan makan suka masyarakat tanah a...      1
6   isu ikan lele kandung sel kanker jejaring sosi...      0
7   heboh berita hoax putar konspirasi kanker arti...      1
8   jejaring sosial edar informasi ikan lele ikan ...      1
9   jejaring sosial edar informasi lele ikan jorok...      0
10  baca baca tulis edar media sosial kena tusuk j...      1
11  metode spesialis jantung rumah sakit harap dr ...      1
12  doktersehat com stroke rupa sakit saraf akibat...      0
13  apple kabar kembang iphone usung ukur layar ka...      1
14  minggu muncul lapor iphone 6 plus mudah lengku...      1
15  apple nyata iphone 6

In [ ]:
testset.to_csv(path_or_buf='/content/testset.csv',index=False)

# Tokenizer the dataset

## Initialize variable for tokenizer

In [ ]:
vocab_size = 3000
embedding_dim = 16
max_length = 4610
trunc_type='pre'
oov_tok = "<OOV>"
pad_type = 'post'

## Train Set

In [ ]:
trainset_x = trainset['berita_preprocessed'].to_list()
trainset_y = trainset['class'].to_numpy()
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(trainset_x)
trainset_x_seq = tokenizer.texts_to_sequences(trainset_x)
trainset_padded = pad_sequences(trainset_x_seq, maxlen=max_length, truncating=trunc_type, padding=pad_type)
print(trainset_padded)

[[  62  200  431 ...    0    0    0]
 [ 928  234  113 ...    0    0    0]
 [1062  155   24 ...    0    0    0]
 ...
 [ 972   24   61 ...    0    0    0]
 [  15  102  101 ...    0    0    0]
 [ 351   24   61 ...    0    0    0]]


In [ ]:
print(trainset_x_seq[5])
print(trainset_y)

[8, 404, 50, 2159, 10, 25, 40, 107, 38, 9, 11, 38, 140, 1, 556, 1501, 284, 40, 1729, 9, 11, 106, 10, 38, 40, 9, 11, 204, 38, 1502, 432, 40, 266, 40, 29, 251, 447, 1, 1302, 251, 271, 447, 266, 40, 251, 1303, 447, 1117, 40, 557, 736, 601, 1304, 447, 40, 42, 40, 653, 899, 1071, 218, 48, 1022, 493, 5, 674, 173, 674, 173, 496, 497, 1, 1412, 826, 151, 737, 244, 50, 10, 10, 25, 40, 107, 11, 675, 325, 266, 40, 1729, 9, 11, 225, 40, 1413, 588, 1414, 1415, 558, 1302, 2160, 1305, 618, 1225, 251, 271, 447, 266, 40, 251, 1303, 447, 1117, 40, 557, 736, 601, 1304, 447, 40, 676, 138, 151, 571, 151, 571, 40, 38, 50, 10, 738, 39, 10, 25, 2161, 40, 525, 217, 9, 257, 271, 976, 271, 1226, 572, 1118, 9, 257, 217, 11, 739, 619, 827, 271, 976, 38, 50, 10, 736, 572, 271, 1226, 2162, 38, 50, 10, 10, 25, 1503, 206, 38, 50, 10, 39, 10, 25, 9, 257, 620, 1306, 38, 9, 11, 38, 11, 140, 556, 134, 1908, 1416, 140, 307, 173, 467, 11, 140, 191, 241, 1, 39, 9, 50, 10, 10, 25, 1027, 50, 50, 10, 9, 11, 16, 34, 68, 432, 10, 

## Validation Set

In [ ]:
valset_x = valset['berita_preprocessed'].to_list()
valset_y = valset['class'].to_numpy()
valset_x_seq = tokenizer.texts_to_sequences(valset_x)
valset_padded = pad_sequences(valset_x_seq, maxlen=max_length, truncating=trunc_type, padding=pad_type)
print(valset_padded)

[[ 254  547  475 ...    0    0    0]
 [   1  165  146 ...    0    0    0]
 [  14  608  236 ...    0    0    0]
 ...
 [ 351   24   61 ...    0    0    0]
 [  47 1276 1566 ...    0    0    0]
 [1089  636  158 ...    0    0    0]]


## Test Set

In [ ]:
testset_x = testset['berita_preprocessed'].to_list()
testset_y = testset['class'].to_numpy()
testset_x_seq = tokenizer.texts_to_sequences(testset_x)
testset_padded = pad_sequences(testset_x_seq, maxlen=max_length, truncating=trunc_type, padding=pad_type)
print(testset_padded)

[[ 78   3   2 ...   0   0   0]
 [  1   3   2 ...   0   0   0]
 [  2  28  16 ...   0   0   0]
 ...
 [ 62 434 363 ...   0   0   0]
 [ 92 820 650 ...   0   0   0]
 [ 31  18 718 ...   0   0   0]]
